In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 加载数据
data = pd.read_csv('../balanced_train_data.csv')

# 分离特征和目标变量
X = data.drop(columns=['churn'])
y = data['churn']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

# 定义参数分布
param_distributions = {
    'n_estimators': [int(x) for x in np.linspace(start=100, stop=500, num=5)],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}
# 初始化随机森林
rf = RandomForestClassifier(random_state=42)

# 随机搜索
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=50,  # 随机采样次数
    scoring='roc_auc',
    cv=5,  # 5折交叉验证
    random_state=42,
    n_jobs=-1
)
random_search_rf.fit(X_train, y_train)

# 输出最佳参数
best_params = random_search_rf.best_params_
print("最佳参数:", best_params)

# 使用最佳参数的模型
best_rf = random_search_rf.best_estimator_

# 性能评估
y_pred = best_rf.predict(X_test)
y_pred_proba = best_rf.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print(f"调优后性能: Accuracy={accuracy:.4f}, F1 Score={f1:.4f}, AUC={auc:.4f}")


最佳参数: {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}
调优后性能: Accuracy=0.8252, F1 Score=0.8329, AUC=0.8976


#贝叶斯优化

KeyError: "['Churn'] not found in axis"

ValueError: Found input variables with inconsistent numbers of samples: [11, 33]

调优后性能: Accuracy=0.8197, F1 Score=0.8297, AUC=0.8892
